In [1]:
# Creates the folder kemenyGPU
!git clone https://github.com/noeliarico/kemenyGPU

fatal: destination path 'kemenyGPU' already exists and is not an empty directory.


In [2]:
execfile('kemenyGPU/v1.py')

FileNotFoundError: ignored

In [ ]:
from numba import cuda
import numpy as np
import math
import time
import logging
logging.getLogger().setLevel(logging.DEBUG)

In [ ]:
om = om12
n = 12
m = 10

In [ ]:
stride = 4000000

# create an array of dimension stride x n (initially empty)
# to store the rankings and factorial representations
d_factoradic = cuda.device_array((stride,n), dtype=np.uint8)

# to store the boolean and factorial representations
d_alternatives = cuda.device_array((stride,n), dtype=np.bool_)

threadsperblock = 256
blockspergrid = math.ceil(stride / threadsperblock)
logging.debug("threadsperblock = {}, blockspergrid = {}, total = {}".format(threadsperblock, blockspergrid, threadsperblock*blockspergrid))

# array of one position to store the best distance
best_dist = np.array([(n*n-n)/2]) # initiallize with upper bound
# move the value to the gpu
d_best_dist = cuda.to_device(best_dist)
# move the value to the gpu
best_ranking = np.array([0], dtype=np.uint64) 
d_best_ranking = cuda.to_device(best_ranking)
# single = False
total = math.factorial(n)

start = time.time()
distance_from_ranking_to_profile_gpu[blockspergrid, threadsperblock](d_factoradic, d_alternatives, om, stride, total, d_best_dist, d_best_ranking)
d_best_dist.copy_to_host(best_dist)
d_best_ranking.copy_to_host(best_ranking)
end = time.time()


logging.info("Execution time: {}".format(end-start))
logging.info(best_dist)
logging.info(best_ranking)